<a href="https://colab.research.google.com/github/MengOonLee/AccountReceivable/blob/main/Workflow/InvoicePayment/Forecasing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Invoice Payment Forecasting

Accounts Receivables (AR) is an important aspect of businesses, it is a credit extended by the company to the client that require payments due within a time period. With B2B transactions increasing in volume and complexity, poor management of AR can lead to to unnecessary expenses and cash flow problems.

Invoice payment forecasting can be used to optimize collection strategies that allows businesses to predict and plan for future customer payments. By analyzing customer payment history, businesses can forecast their future accounts receivable balance and plan accordingly. By identifying invoices that are likely to be paid late, customers who are likely to have large amounts of overdue receivables, which in turn can lead to better visibility into future cash flow.

The invoices dataset contains the past payment information and behaviour of various buyers.

1. Categorize the invoice into different aging buckets. The different buckets will be:
   * \< 1 month
   * 1 month - 2 months
   * 2 month - 3 months
   * \> 3 months
  
2. Machine Learning model will predict the payment of an invoice falls into the buckets.

In [1]:
%%bash
pip install --no-cache-dir -qU \
    transformers datasets bitsandbytes peft
pip check

No broken requirements found.


In [ ]:
# Import libraries 
import argparse
import bitsandbytes as bnb
from datasets import load_dataset
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset

# Reproducibility
seed = 42
set_seed(seed)

## Preprocessing

List of the fields in dataset are as follows:
* Customer: Customer code of the account
* Country: customer country
* Region:
* Account Owner Name
* Last Payment Date
* Contact Date
* Collection Specialist
* Projected Overdue Amount
* Current Overdue Amount
* Collected
* Forecast
* Forecast Status

In [2]:
# Load the large language model file
from llama_cpp import Llama
LLM = Llama(model_path="./llama-2-7b-chat.ggmlv3.q8_0.bin")

prompt = "Q: Why are Jupyter notebooks difficult to maintain? A:"

output = LLM(prompt, max_tokens=0)

print(output["choices"][0]["text"])

In [ ]:
import os
import pandas as pd

input_dir = '../../Data/Input/'
input_payment = 'Payment.csv'
df_payment = pd.read_csv(os.path.join(input_dir, input_payment))

df_payment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   business_code           50000 non-null  object 
 1   cust_number             50000 non-null  object 
 2   name_customer           50000 non-null  object 
 3   clear_date              45181 non-null  object 
 4   buisness_year           50000 non-null  float64
 5   doc_id                  50000 non-null  float64
 6   posting_date            50000 non-null  object 
 7   document_create_date    50000 non-null  int64  
 8   document_create_date.1  50000 non-null  int64  
 9   due_in_date             50000 non-null  float64
 10  invoice_currency        50000 non-null  object 
 11  document type           50000 non-null  object 
 12  posting_id              50000 non-null  float64
 13  area_business           0 non-null      float64
 14  total_open_amount       50000 non-null

In [ ]:
import xgboost as xgb